In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer

/Users/arvind/miniconda3/envs/treehacks24/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
encoder = SentenceTransformer('all-mpnet-base-v2')

In [3]:
classes = ["clarify", "email", "link", "schedule", "unknown"]

In [4]:
# Walk through all files in a directory
import os
lines = []
for class_ in classes:
    # Read contents of file
    with open('data/action_classification/' + class_ + '.txt', 'r') as f:
        lines += f.readlines()

inputs = encoder.encode(lines)
inputs = torch.Tensor(inputs)
print(inputs.shape)

torch.Size([500, 768])


In [5]:
outputs = []
for class_ in classes:
    output_1he = torch.Tensor([0] * len(classes))
    output_1he[classes.index(class_)] = 1
    for i in range(100):
        outputs.append(output_1he)

outputs = torch.stack(outputs)
print(outputs.shape)

torch.Size([500, 5])


In [6]:
# Split data
from sklearn.model_selection import train_test_split
inputs_train, inputs_test, outputs_train, outputs_test = train_test_split(inputs, outputs, test_size=0.2, random_state=42)

In [7]:
class ActionClassifier(torch.nn.Module):
    def __init__(self):
        super(ActionClassifier, self).__init__()
        self.fc1 = nn.Linear(768, 384)
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(384, 384)
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(384, 384)
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(384, 384)
        self.dropout4 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(384, len(classes))
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = F.gelu(self.fc1(x))
        x = self.dropout1(x)
        x = F.gelu(self.fc2(x))
        x = self.dropout2(x)
        x = F.gelu(self.fc3(x))
        x = self.dropout3(x)
        x = F.gelu(self.fc4(x))
        x = self.dropout4(x)
        x = F.gelu(self.fc5(x))
        x = self.softmax(x)
        return x

model = ActionClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [8]:
best_test_loss = float('inf')
for epoch in range(5000):
    optimizer.zero_grad()
    preds_train = model(inputs_train)
    train_loss = criterion(preds_train, outputs_train)
    train_loss.backward()
    train_acc = (preds_train.argmax(1) == outputs_train.argmax(1)).float().mean()
    optimizer.step()

    preds_test = model(inputs_test)
    test_loss = criterion(preds_test, outputs_test)
    test_acc = (preds_test.argmax(1) == outputs_test.argmax(1)).float().mean()
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'server/action_classifier.pt')

    if epoch % 100 == 0:
        print('Epoch {} train loss: {} train acc: {} test loss: {} best test loss: {} test acc: {}'.format(epoch, train_loss, train_acc, test_loss, best_test_loss, test_acc))

Epoch 0 train loss: 1.6092833280563354 train acc: 0.2150000035762787 test loss: 1.6100573539733887 best test loss: 1.6100573539733887 test acc: 0.14000000059604645
Epoch 100 train loss: 1.4280565977096558 train acc: 0.4449999928474426 test loss: 1.5340031385421753 best test loss: 1.5340031385421753 test acc: 0.27000001072883606
Epoch 200 train loss: 1.0811355113983154 train acc: 0.8450000286102295 test loss: 1.1846754550933838 best test loss: 1.1846754550933838 test acc: 0.75
Epoch 300 train loss: 0.9466697573661804 train acc: 0.9599999785423279 test loss: 1.0640063285827637 best test loss: 1.058962345123291 test acc: 0.8500000238418579
Epoch 400 train loss: 0.9405715465545654 train acc: 0.9649999737739563 test loss: 1.0655317306518555 best test loss: 1.0533347129821777 test acc: 0.8299999833106995
Epoch 500 train loss: 0.9352471232414246 train acc: 0.9700000286102295 test loss: 1.0620434284210205 best test loss: 1.0394728183746338 test acc: 0.8399999737739563
Epoch 600 train loss: 0.9

In [9]:
action_classifier = ActionClassifier()
action_classifier.load_state_dict(torch.load('server/action_classifier.pt'))
action_classifier.eval()

ActionClassifier(
  (fc1): Linear(in_features=768, out_features=384, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=384, out_features=384, bias=True)
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=384, out_features=384, bias=True)
  (dropout3): Dropout(p=0.2, inplace=False)
  (fc4): Linear(in_features=384, out_features=384, bias=True)
  (dropout4): Dropout(p=0.2, inplace=False)
  (fc5): Linear(in_features=384, out_features=5, bias=True)
  (softmax): Softmax(dim=1)
)

In [10]:
def infer(text):
    input_ = torch.Tensor(encoder.encode([text]))
    output = action_classifier(input_)
    print(output)
    return classes[output.argmax(1)]

In [11]:
infer('Let\'s all meet sometime to discuss sprint planning for this project. Who\'s available?')

tensor([[1.4797e-07, 2.1970e-04, 1.4797e-07, 9.9978e-01, 1.4797e-07]],
       grad_fn=<SoftmaxBackward0>)


'schedule'